In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
# We'll start by reading the 'FederallyInsuredCreditUnions_2024q1.csv' file into a pandas dataframe 

df = pd.read_csv('your_path/FederallyInsuredCreditUnions_2024q1.csv')

# Then get an idea of the size and shape of the dataframe
df_shape_initial = print(f"Initial dataframe is {df.shape[1]} columns wide and {df.shape[0]} rows deep.")
df_shape_initial


In [ ]:
# View column names along with their indexes for future reference. 

# Note: some column names break into a second line due to the presence of newline characters ('\n'), so we'll remove those characters first, making the column names more readable.

df.columns = [col.replace('\n', ' ') for col in df.columns]
for idx, col in enumerate(df.columns):
    print(f"{idx}: {col}")

In [ ]:
# View top and bottom 5 rows to get a sense for the type of data, its completeness, and overall composition.

# We'll need to adjust Jupyter's display settings to see all 26  first.
pd.set_option('display.max_columns', None)

# Then we can use 'display' to generate a neat and readable output table.
top_bottom = pd.concat([df.head(5), df.tail(5)])
display(top_bottom)



In [ ]:
# Based on output, we'll adress the following before proceeding with the analysis. 
# Drop the bottom two rows of containing unecessary information.
# Explore the second-to-last column and possibly drop if populated with all nulls.
# Correct data types. 


In [ ]:
# 1. Drop Bottom Two Rows

# Drop the rows and then have another look at the bottom 5 rows to confirm.
df.drop(df.tail(2).index, inplace=True)
display(df.tail(5))

# We see the last row is at index 4571, "SESLOC" CU, and the two records at indexes 4572 and 4573 have been dropped. 

In [ ]:
# 2. Drop Null Column
# I suspect the second-to-last column has only null values. 

#We'll first confirm if that's the case, along with a descriptive 'print' command for easily readable output.

all_nulls = df.iloc[:, 24].isnull().all()
column_name = df.columns[24]
print(f"Are all values in the column '{column_name}' null?: {all_nulls}")

In [ ]:
# Seeing they're all nulls, I'll drop the whole column at index 24 and take another look at the bottom rows.
df = df.drop(df.columns[24], axis=1)
df_shape_new = print(f"New dataframe is {df.shape[1]} columns wide and {df.shape[0]} rows deep.\n")
df_shape_new
display(df.tail(5))

In [ ]:
# 3. Correcting Data Types

# Trying to correct the data types with NaN values in the dataframe can be complicated. 

# I'll cover properly addressing null values a bit later. But to move ahead with correcting data types now, I'm going to use a three-step approach.
# First, create a temporary df where I'll convert the null values to non-null values.
# Second, convert the data types. 
# Third, replace the nulls so we can address them properly a bit later.  



In [ ]:
# Step 1 - Temporarily Replace Nulls

# Let's start by viewing the current data types
print(df.dtypes)


In [ ]:
# Notice discrepancies in data types, e.g.,'Year' should not be a float64 dtype, and 'Members' should not be an object dtype.


In [ ]:
# Step 1a. Create a Copy of the Dataframe

# A copy of the dataframe allows us to convert null values while still preserving them in the original dataframe. 
df_temp = df.copy()

# Print column names to ensure we enter them exactly as they're presented in the dataframe (e.g. characters, whitespace, etc.). 
print(df.columns.tolist())

In [ ]:
# Step 1b. Convert Null Values in Temp Dataframe

# Replace null values in numeric columns of the temp dataframe with 0.

# Let's declare and initialize a resuable variable for numeric columns
numeric_columns = ['Charter number', 'Members', 'Total assets', 'Total loans',
                   'Total deposits', 'Total deposits, 4 quarter growth  (%)',
                   'Total loans,  4 quarter growth  (%)', 'Total assets,  4 quarter growth  (%)',
                   'Members, 4 quarter growth  (%)', 'Net worth,  4 quarter growth (excludes CECL transition provision) (%)']

# Now replace invalid values with 0 in the temp dataframe. Note: We'll need to replace several invalid values such as 'NA', 'N/M', etc., in addition to nulls.
for col in df_temp:
    df_temp[col] = df_temp[col].replace(['NA','N/M', 'N/M - Not Meaningful','N/M - Not Meaningful ', np.nan], 0)  

In [ ]:
# Verify there are no null values in any records.

null_indexes = df_temp[df_temp.isnull().any(axis=1)].index.tolist()
print("Indexes of rows with null values:", null_indexes)

# Having converted all the null values in the temporary dataframe, we can now correct our data types. 

In [ ]:
# Step 2 - Correct the Data Types in Temp Dataframe

# I'll create a reusable function that takes a parameter 'data' which can be any dataframe with the same structure, and performs the required data type conversions.

def convert_dataframe_types(data):
    # Convert columns to int
    int_columns = [
        'Year', 'Quarter ', 'NCUA region', 'Charter number', 
        'NCUA internal ID (join_number)'
    ]
    for col in int_columns:
        if col in data.columns:
            data[col] = data[col].astype(int)
    
    # Convert columns to float
    float_columns = [
        'Total deposits, 4 quarter growth  (%)', 
        'Total loans,  4 quarter growth  (%)', 
        'Total assets,  4 quarter growth  (%)', 
        'Members, 4 quarter growth  (%)', 
        'Net worth,  4 quarter growth (excludes CECL transition provision) (%)'
    ]
    for col in float_columns:
        if col in data.columns:
            data[col] = data[col].astype(float)
    
    # Convert Zip code to 5-digit string
    zip_code_column = 'Zip code (Mailing address)'
    if zip_code_column in data.columns:
        data[zip_code_column] = data[zip_code_column].replace([np.nan], 0).astype(int).astype(str).str.zfill(5)
    
    # Remove commas and convert to int
    comma_int_columns = ['Members', 'Total assets', 'Total loans', 'Total deposits']
    for col in comma_int_columns:
        if col in data.columns:
            data[col] = data[col].str.replace(',', '').astype(int)
    
    return data

# Usage with df_temp
df_temp = convert_dataframe_types(df_temp)


In [ ]:
# Verify the data types have been properly updated.
print(df_temp.dtypes)
display(df_temp.head(3))

In [ ]:
# Step 3 - Decide How to Handle Null Values in Original Dataframe

# If none of the null-containing records belong to the top 5 credit unions as measured by 'Members', 'Total assets', 'Total loans', or 'Total deposits', we will drop them.


In [ ]:
# Step 3a. Get a list of records with null values

# Create a variable containing the indexes of rows with null values in the original dataframe. 
null_indexes_df = df[df.isnull().any(axis=1)].index
print("Indexes of rows with null values in original df:", null_indexes_df.tolist())

    

In [ ]:
# 3b. Use the corresponding null-free records in the temp dataframe to decide appropriate resolution

# We first have to define our target groups so we can then check if they include any records from null_indexes_df.

# Assign variables for the top 5 groups.
top_5_members = df_temp.nlargest(5, 'Members').index
top_5_assets = df_temp.nlargest(5, 'Total assets').index
top_5_loans = df_temp.nlargest(5, 'Total loans').index
top_5_deposits = df_temp.nlargest(5, 'Total deposits').index

# Combine the "top 5" indexes.
top_indexes = set(top_5_members).union(set(top_5_assets)).union(set(top_5_loans)).union(set(top_5_deposits))

# Assign a variable to check for intersections between null_indexes_df and top_indexes.
impactful_null_rows = set(null_indexes_df).intersection(top_indexes)

# Then check to see if there are any impactful null records.
if impactful_null_rows:
    print("These rows with null values are among the top 5 in Members, Total assets, or Total loans:", impactful_null_rows)
else:
    print("No rows with null values are among the top 5 in Members, Total assets, or Total loans.")


In [ ]:
# Based on the output, we can safely drop the null-containing records in the original dataframe.

In [ ]:
# There are two important steps we performed on the temporary dataframe, which we now need to perform on the original dataframe.
# Those are: dropping the null-containing rows and correcting the data types.

In [ ]:
# First, we drop the non-impactful null rows.
if not impactful_null_rows:
    df = df.drop(null_indexes_df)
    print("Dropped rows with null values. Remaining rows:", df.shape[0])
else:
    print("Consider handling the nulls in the identified top rows instead of dropping.")



In [ ]:
# The 11 records have been dropped leaving us with 4561 rows in the original dataframe vs the original 4572.

# We'll now convert the data types in the original dataframe.

# First confirm the current data types and format of the dataframe.
print(df.dtypes)
display(df.head(3))

In [ ]:
# Notice some data type discrepancies, e.g., 'Year' as float64, and Zip codes as float64.

# Use the convert_dataframe_types(data) function we created earlier and pass 'df' as the parameter.
# Executing the function and looking once again at the data types and dataframe confirms that it's been done.
convert_dataframe_types(df)
print(df.dtypes)
display(df.head(3))

In [ ]:
# This concludes the initial cleanup of our dataframe. We'll have a much easier time conducting further analysis by effectively having removed potential issues and complexity, and gained familiarity with the structure and contents of the dataframe.
# We'll save this df as a csv so we can easily reference or share it later.

df.to_csv('cleaned_credit_union_data.csv', index=False)